<a href="https://colab.research.google.com/github/Nargiz-chess/Solar_forecasting/blob/main/AUS_Lridge_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [197]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

from pandas import read_csv
import pandas as pd
import datetime as dt
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statistics import mean
from sklearn import metrics


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [198]:
path1 = '/content/drive/MyDrive/Master_thesis/Datasets/Australia_numpy_removed_night_hours.npy'
path2 = '/content/drive/MyDrive/Master_thesis/Datasets/irradiance_AUS(2012-2013).csv'

In [199]:
weath_data=read_csv(path2, header =0, index_col=0) 
weath_data=weath_data.drop("time", axis=1)
weath_data=weath_data.values
weath_data = weath_data.reshape(weath_data.shape[0]*weath_data.shape[1])
weath_data.shape

(6570,)

In [200]:
array_aus = np.loadtxt(path1)
print("Shape before reshape",np.shape(array_aus))


Shape before reshape (6570, 299)


In [201]:
###explarotary

# df_AUS = pd.DataFrame(array_aus)
# df_AUS.head(25)

### note: rows time steps, columns houses. 0-th row means at 5am (night hours between 22h and 5h were removed)

#Random houses - subsetting

In [202]:
import random

random.seed(42)

subset_col=[]
while True:
  r=random.randint(0,298)
  if r not in subset_col:
    subset_col.append(r)
  if len(subset_col) ==40:
    break

array_aus = array_aus[:,subset_col]
print(array_aus.shape)

(6570, 40)


#Last 5 days training

In [203]:
array_5days = array_aus[-(6*18):,:]
#weather = df_weather.fillna(0)
weath_data_5days = weath_data[-108:].reshape(-1,1)
print("Shape of solar power dataset =",array_5days.shape, "\n", "Shape of weather data",weath_data_5days.shape)


Shape of solar power dataset = (108, 40) 
 Shape of weather data (108, 1)


#L-ridge

In [204]:
import numpy as np
from scipy.ndimage.interpolation import shift
from sklearn.linear_model import Ridge

def shifting(X,y,n_shift):
  y = y.flatten()
  y_shift = shift(y,-n_shift)
  y_shift = y_shift[:-n_shift]
  X_shift = X[:-n_shift]
  return X_shift,y_shift

In [205]:
RMSEs = []

for house in range(40):
  X = np.concatenate((weath_data_5days.reshape(-1,1), array_5days[:,house].reshape(-1,1)), axis=1) 
  y=array_5days[:,house]
  X,y = shifting(X,y,18)
  y = y.reshape(-1,1)
  #print(y.shape)
  X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=18, shuffle = False)
  reg = Ridge(alpha=0.1, fit_intercept=True).fit(X_train, y_train)

  y_pred = reg.predict(X_test)
  rmse = np.sqrt(metrics.mean_squared_error(y_test,y_pred))
  RMSEs.append(rmse)

print("Average RMSE = ", mean(RMSEs))

Average RMSE =  0.13625437052764375
